In [6]:
import torch
import torchvision
import os

try:
  from google.colab import drive
  drive.mount('/content/drive')

  os.chdir('/content/drive/MyDrive/GitHub/kornia')
except Exception:   # Not run on Google Colab
  import getpass
  if getpass.getuser() == 'vscode':       # Running in container
    os.chdir('/workspaces/kornia/')
  else:                                   # Running in virtual environment
    os.chdir(os.path.join(os.path.expanduser('~'), 'Documents/Git/kornia/'))

print(os.getcwd())
import sys
sys.path.append(os.getcwd())
cwd_path = os.getcwd()

from kornia.geometry.nerf.camera_utils import CameraParser, create_cameras_at_origin, initialize_cameras_at_origin, \
                                              cameras_for_ids
from kornia.geometry.nerf.nerf_solver import NerfSolver
from torchvision.io import read_image
import matplotlib.pyplot as plt
import glob

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/kornia


## Load scene images and plot one of them

In [7]:
img_paths = glob.glob('kornia/geometry/nerf/kornia_nerf_data/flower_4/images/*.png')

transform = torchvision.transforms.Resize((250))    # In case we want to reduce resolution
imgs: torch.tensor = []
camera_ids = []
num_cams_to_load = -1
for i, img_path in enumerate(img_paths):
    if num_cams_to_load != -1 and i == num_cams_to_load:
        break
    img = read_image(img_path)
    # img = transform(img)  # Uncomment to apply downsizing
    imgs.append(img)
    camera_ids.append(i)

print(len(imgs))
img_plt = torch.permute(imgs[0], (1, 2, 0))
#plt.imshow(img_plt)
img_plt.shape

33


torch.Size([756, 1008, 3])

## Parse cameras from estimated pos xml

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

camera_parser = CameraParser('./test/geometry/nerf/cameras.xml', device=device)
cameras = camera_parser.create_cameras()

# cameras = create_cameras_at_origin(cameras) # Put all cameras at origin, facing positive z-axis
# cameras = initialize_cameras_at_origin(cameras) # Initialize all cameras at origin, focals equals image size
# cameras = cameras_for_ids(cameras, camera_ids)  # To use same subset of cameras as input images

print(cameras.batch_size)

cuda
33


## Run training on image scene

In [11]:
nerf_obj = NerfSolver(device=device)
num_img_rays = 1024
batch_size = 1024
nerf_obj.init_training(cameras=cameras, min_depth=1., max_depth=2., ndc=True, imgs=imgs, num_img_rays=num_img_rays, 
                       batch_size=batch_size, num_ray_points=128)

In [ ]:
torch.manual_seed(0)    # For reproducibility of random processes

nerf_obj.run(num_epochs=10000)

Epoch: 0: epoch_psnr = 11.864176461675225
Epoch: 10: epoch_psnr = 13.337646762558467
Epoch: 20: epoch_psnr = 13.382343196016429
Epoch: 30: epoch_psnr = 13.49877445955691
Epoch: 40: epoch_psnr = 13.681131354427773
Epoch: 50: epoch_psnr = 13.925966088807524


## Rendering novel views

### Sub-set of input cameras

In [ ]:
from kornia.geometry.nerf.camera_utils import cameras_for_ids

camera_ids = [2]
cameras_to_render = cameras_for_ids(cameras, camera_ids)

In [ ]:
imgs_rendered = nerf_obj.render_views(cameras_to_render)

In [ ]:
#plt.imshow(imgs_rendered[0])

### Spiral pattern

In [ ]:
from kornia.geometry.nerf.camera_utils import create_spiral_path

cameras_spiral = create_spiral_path(cameras, rad=1., num_views=20, num_circles=2)
imgs_rendered_spiral = nerf_obj.render_views(cameras_spiral)

In [ ]:
import numpy as np
import imageio

imgs_rendered_spiral_numpy = [img_rendered_spiral.cpu().numpy() for img_rendered_spiral in imgs_rendered_spiral]

dir = 'kornia/geometry/nerf/kornia_nerf_data/flower_4'
scene_name = 'flower_4'
gif_path = os.path.join(dir, scene_name + '_spiral.gif')
imageio.mimwrite(gif_path, imgs_rendered_spiral_numpy, fps=30)

In [ ]:
from IPython.display import Image
#Image(open(gif_path, 'rb').read())